# Fetching new data from 2024 12 08

In [1]:
import os
import ccxt
import pandas as pd
from datetime import datetime
api_key = os.getenv('BINANCE_API_KEY')
secret_key = os.getenv('BINANCE_SECRET_KEY')
exchange = ccxt.binance({
    'apiKey': api_key,
    'secret': secret_key,
    'enableRateLimit': True,
    'options': {
        'defaultType': 'spot',
        'test': True  
    },
})
symbol = 'BTC/USDT'
timeframe = '1m'  
start_time = '2024-08-12T00:00:00Z'
end_time = datetime.utcnow().isoformat()  
start_timestamp = exchange.parse8601(start_time)
end_timestamp = exchange.parse8601(end_time)
def fetch_ohlcv(symbol, timeframe, since, end_timestamp, limit=1000):
    ohlcv = []
    while since < end_timestamp:
        try:
            candles = exchange.fetch_ohlcv(symbol, timeframe, since, limit)
            if not candles:
                break
            ohlcv.extend(candles)
            since = candles[-1][0] + 1 
            print(f"Fetched {len(candles)} candles, last timestamp: {pd.to_datetime(candles[-1][0], unit='ms')}")
        except ccxt.NetworkError as e:
            print(f"Network error: {e}")
            break
        except ccxt.ExchangeError as e:
            print(f"Exchange error: {e}")
            break
        except Exception as e:
            print(f"Unexpected error: {e}")
            break
    return ohlcv

ohlcv_data = fetch_ohlcv(symbol, timeframe, start_timestamp, end_timestamp)

df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
df.set_index('timestamp', inplace=True)
df.to_csv('btc_1m_data.csv')

Fetched 1000 candles, last timestamp: 2024-08-12 16:39:00
Fetched 1000 candles, last timestamp: 2024-08-13 09:19:00
Fetched 1000 candles, last timestamp: 2024-08-14 01:59:00
Fetched 1000 candles, last timestamp: 2024-08-14 18:39:00
Fetched 1000 candles, last timestamp: 2024-08-15 11:19:00
Fetched 1000 candles, last timestamp: 2024-08-16 03:59:00
Fetched 1000 candles, last timestamp: 2024-08-16 20:39:00
Fetched 1000 candles, last timestamp: 2024-08-17 13:19:00
Fetched 1000 candles, last timestamp: 2024-08-18 05:59:00
Fetched 1000 candles, last timestamp: 2024-08-18 22:39:00
Fetched 1000 candles, last timestamp: 2024-08-19 15:19:00
Fetched 1000 candles, last timestamp: 2024-08-20 07:59:00
Fetched 1000 candles, last timestamp: 2024-08-21 00:39:00
Fetched 1000 candles, last timestamp: 2024-08-21 17:19:00
Fetched 1000 candles, last timestamp: 2024-08-22 09:59:00
Fetched 1000 candles, last timestamp: 2024-08-23 02:39:00
Fetched 1000 candles, last timestamp: 2024-08-23 19:19:00
Fetched 1000 c

# Multiple indicators

In [2]:
import pandas as pd
import numpy as np
import sqlite3
from apscheduler.schedulers.blocking import BlockingScheduler

df = pd.read_csv('btc_1m_data.csv', parse_dates=['timestamp'])
df.set_index('timestamp', inplace=True)

In [3]:


def resample_signals(df):
    # Super Trend
    df['ATR'] = df['high'] - df['low']
    df['ATR'] = df['ATR'].rolling(window=14).mean()
    multiplier = 3
    df['basic_upper_band'] = (df['high'] + df['low']) / 2 + multiplier * df['ATR']
    df['basic_lower_band'] = (df['high'] + df['low']) / 2 - multiplier * df['ATR']
    df['Super_Trend'] = df['basic_upper_band'].where(df['close'] > df['basic_upper_band'].shift(1),
                                                     df['basic_lower_band'].where(df['close'] < df['basic_lower_band'].shift(1)))
    df['Super_Trend'].ffill(inplace=True)
    df['Super_Trend_signal'] = np.where(df['close'] > df['Super_Trend'], 1, -1)

    # EMA
    df['EMA'] = df['close'].ewm(span=20, adjust=False).mean()
    df['EMA_signal'] = np.where(df['close'] > df['EMA'], 1, -1)

    # Hybrid Indicator
    donchian_period = 20
    df['Donchian_High'] = df['high'].rolling(window=donchian_period).max()
    df['Donchian_Low'] = df['low'].rolling(window=donchian_period).min()
    df['cumulative_volume'] = df['volume'].cumsum()
    df['cumulative_price_volume'] = (df['close'] * df['volume']).cumsum()
    df['VWAP'] = df['cumulative_price_volume'] / df['cumulative_volume']
    moving_average_period = 50
    df['Moving_Avg'] = df['close'].rolling(window=moving_average_period).mean()
    df['Breadth_Indicator'] = np.where(df['close'] > df['Moving_Avg'], 1, 0)
    
    df['Hybrid_signal'] = np.where(
        (df['close'] > df['Donchian_High'].shift(1)) & (df['close'] > df['VWAP']) & (df['Breadth_Indicator'] == 1), 1,
        np.where(
            (df['close'] < df['Donchian_Low'].shift(1)) & (df['close'] < df['VWAP']) & (df['Breadth_Indicator'] == 0), -1, np.nan
        )
    )
    df['Hybrid_signal'].fillna(-1, inplace=True)
    df['Hybrid_signal'] = df['Hybrid_signal'].astype(int)

    # Stochastic Oscillator
    n = 14
    df['Lowest_Low'] = df['low'].rolling(window=n).min()
    df['Highest_High'] = df['high'].rolling(window=n).max()
    df['%K'] = ((df['close'] - df['Lowest_Low']) / (df['Highest_High'] - df['Lowest_Low'])) * 100
    df['%D'] = df['%K'].rolling(window=3).mean()

    df['Stochastic_signal'] = np.where(
        (df['%K'] > df['%D']) & (df['%K'].shift(1) <= df['%D'].shift(1)), 1,
        np.where(
            (df['%K'] < df['%D']) & (df['%K'].shift(1) >= df['%D'].shift(1)), -1, np.nan
        )
    )
    df['Stochastic_signal'].fillna(-1, inplace=True)
    df['Stochastic_signal'] = df['Stochastic_signal'].astype(int)

    # Bollinger Bands
    window = 20
    no_of_std = 2
    df['MA'] = df['close'].rolling(window=window).mean()
    df['BB_Upper'] = df['MA'] + (df['close'].rolling(window=window).std() * no_of_std)
    df['BB_Lower'] = df['MA'] - (df['close'].rolling(window=window).std() * no_of_std)
    df['BB_signal'] = np.where(df['close'] < df['BB_Lower'], 1, np.where(df['close'] > df['BB_Upper'], -1, np.nan))
    df['BB_signal'].fillna(-1, inplace=True)
    df['BB_signal'] = df['BB_signal'].astype(int)
    resampled_df = df.resample('4H').agg({
        'Super_Trend_signal': 'last',
        'EMA_signal': 'last',
        'Hybrid_signal': 'last',
        'Stochastic_signal': 'last',
        'BB_signal': 'last'
    }).dropna()

    return resampled_df
resampled_df = resample_signals(df)
print(resampled_df)


                     Super_Trend_signal  EMA_signal  Hybrid_signal  \
timestamp                                                            
2024-08-12 00:00:00                   1          -1             -1   
2024-08-12 04:00:00                   1           1             -1   
2024-08-12 08:00:00                   1           1             -1   
2024-08-12 12:00:00                   1           1              1   
2024-08-12 16:00:00                  -1           1             -1   
...                                 ...         ...            ...   
2024-08-27 12:00:00                  -1           1             -1   
2024-08-27 16:00:00                  -1           1             -1   
2024-08-27 20:00:00                  -1          -1             -1   
2024-08-28 00:00:00                   1           1             -1   
2024-08-28 04:00:00                  -1           1             -1   

                     Stochastic_signal  BB_signal  
timestamp                            

C:\Users\MTariq\AppData\Local\Temp\ipykernel_16852\394797738.py:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Super_Trend'].ffill(inplace=True)
C:\Users\MTariq\AppData\Local\Temp\ipykernel_16852\394797738.py:34: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, whe

# Insterting signals to the DB

In [4]:
import pandas as pd
import sqlite3
def convert_timestamps(df):
    df.reset_index(inplace=True)
    df['timestamp'] = df['timestamp'].astype(str)
    return df
conn = sqlite3.connect('indicators.db')
cursor = conn.cursor()
cursor.execute('''
CREATE TABLE IF NOT EXISTS Indicators (
    date_time TEXT PRIMARY KEY,
    Super_Trend_signal INTEGER,
    EMA_signal INTEGER,
    Hybrid_signal INTEGER,
    Stochastic_signal INTEGER,
    BB_signal INTEGER
)
''')
def insert_data(df):
    df = convert_timestamps(df) 
    data = df.values.tolist()
    cursor.executemany('''
    INSERT OR REPLACE INTO Indicators (date_time, Super_Trend_signal, EMA_signal, Hybrid_signal, Stochastic_signal, BB_signal)
    VALUES (?, ?, ?, ?, ?, ?)
    ''', data)
    conn.commit()
insert_data(resampled_df)
conn.close()


In [2]:
import pandas as pd
import numpy as np
import sqlite3
from prophet.serialize import model_from_json

# Load data
df = pd.read_csv('btc_1m_data.csv', parse_dates=['timestamp'])
df.set_index('timestamp', inplace=True)

# Load Prophet Model
with open('prophet_model.json', 'r') as f:
    prophet_model = model_from_json(f.read())

# Function to generate predictions and signals
def generate_fb_signals(df):
    # Prepare data for Prophet
    prophet_df = df.reset_index()[['timestamp', 'close']].rename(columns={'timestamp': 'ds', 'close': 'y'})

    # Predict the future using the Prophet model
    future = prophet_model.make_future_dataframe(periods=0, freq='1min')
    forecast = prophet_model.predict(future)

    # Merge predictions with the original data
    prophet_df = prophet_df.merge(forecast[['ds', 'yhat']], on='ds', how='left')
    prophet_df.set_index('ds', inplace=True)

    # Check the first few rows to see the actual vs predicted values
    print("Actual vs Predicted values:")
    print(prophet_df[['y', 'yhat']].head(10))  # Adjust this as needed to view more values

    # Generate Buy (1) and Sell (-1) signals
    prophet_df['FB_signal'] = np.where(prophet_df['y'] > prophet_df['yhat'], 1, -1)

    # Check signals generated
    print("Generated Signals:")
    print(prophet_df['FB_signal'].value_counts())  # Check the distribution of 1 and -1

    # Resample to 4-hour intervals, taking the last signal within each interval
    resampled_signals = prophet_df['FB_signal'].resample('4H').last().dropna()

    return resampled_signals

# Generate signals
fb_signals = generate_fb_signals(df)

# Function to check if a column exists in the SQLite table
def column_exists(cursor, table_name, column_name):
    cursor.execute(f"PRAGMA table_info({table_name})")
    columns = [info[1] for info in cursor.fetchall()]
    return column_name in columns

# Function to append signals to SQLite database
def append_signals_to_db(signals, db_name='indicators.db', table_name='indicators', column_name='FB_signal'):
    # Connect to the SQLite database
    conn = sqlite3.connect(db_name)
    cursor = conn.cursor()

    # Check if the column exists, if not, add it
    if not column_exists(cursor, table_name, column_name):
        cursor.execute(f"ALTER TABLE {table_name} ADD COLUMN {column_name} INTEGER")

    # Iterate over signals and update the database
    for timestamp, signal in signals.items():
        # Convert Timestamp to string format 'YYYY-MM-DD HH:MM:SS' to match the 'date_time' column format
        timestamp_str = timestamp.strftime('%Y-%m-%d %H:%M:%S')
        cursor.execute(f"""
        UPDATE {table_name}
        SET {column_name} = ?
        WHERE date_time = ?
        """, (signal, timestamp_str))

    # Commit changes and close the connection
    conn.commit()
    conn.close()

# Append signals to the database
append_signals_to_db(fb_signals)


Actual vs Predicted values:
                            y  yhat
ds                                 
2024-08-12 00:00:00  58722.99   NaN
2024-08-12 00:01:00  58675.56   NaN
2024-08-12 00:02:00  58720.01   NaN
2024-08-12 00:03:00  58796.29   NaN
2024-08-12 00:04:00  58767.12   NaN
2024-08-12 00:05:00  58810.51   NaN
2024-08-12 00:06:00  58817.99   NaN
2024-08-12 00:07:00  58772.72   NaN
2024-08-12 00:08:00  58799.84   NaN
2024-08-12 00:09:00  58803.98   NaN
Generated Signals:
FB_signal
-1    23416
Name: count, dtype: int64


C:\Users\MTariq\AppData\Local\Temp\ipykernel_1536\533212131.py:39: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  resampled_signals = prophet_df['FB_signal'].resample('4H').last().dropna()
